# Abstract
**A norvel Parsing Induced Learner to exploit parsing information to assist pose estimation, The proposed Parsing Induced Learner is composed of a parsing encoder andd a pose model parameter adapter**    
** 所以重点是 Parsing Induced Learner**     
# The Proposed Approach
## 3.1 The Formulation     
Existing works suggest applying the parsing map S in pose processing to refine the pose estimation P,**Differently, we propose a generic parsing induced pose estimation model $f_{[\theta, \theta']}$ to fully leverage parsinginformation learned form the pair (I,S) in a flexible and effective way :**     
&emsp; $f_{[\theta,\theta']}:I\to P, where\ \theta' = g(I,S)$&emsp;(1)     
上式中$I是原图像输入，S是parsing\ map， P是人体关节$,所以上式意味着$由I和S决定的g函数决定参数\theta'，然后基于参数\theta'（该参数是动态的，所以能很好的调整模型从而捕捉到更好的信息）与原始的输入，决定函数f，从而正确预测I到P的模型$。    
此外，上式中的g函数由提取解析特征的解析编码器${E_{\theta}^S}_S(\cdot)$和学习动态参数$\theta'$的参数适应器$K_{\phi}(\cdot)$组成:    
&emsp; $ \theta' = g(I,S):=K_{\phi}({E_{\theta}^S}_S(I))$&emsp;(2)     
on top of these two encoders are pose and parsing classifiers ${C_w^P}_P(\cdot)$ and ${C_w^S}_S(\cdot)$(猜测，$K_{\phi}可能就是C_W^S$),此外$w^p$和$\theta^p$ 共同实例化了（1）式中的参数$\theta$。最终的损失函数为：    
    <img src="img/loss.png" width=320 height=80>
## 3.2 The Network Architecture
### Pose Encoder    
两种选择，基于VGG16或者基于Hourglass network提取$F^P$.
### Parsing Encoder      
提取特征$F^S = {E_{\theta}^S}_S(\cdot)$，是PIL的前半部分，同样有两种实现方式，VGG16和Hourglass network.    
### Parameter Adapter       
$K_{\phi}(\cdot)$是PIL的后半部分，用来基于$F^S$预测$\theta'$,其结构如下：    
    <img src="img/adapter.png" width=480 height=320>
**由于使用多层CNN预测$\theta'$的参数空间过大，且由于参数较多，很容易造成过拟合以及浪费时间，所以，为了避免这个结果，我们使用一种类似于SVD的方式分解，减少自由参数：**    
&emsp;$ \theta' = U * \tilde\theta *_c V$     
其中U和V是从adaptive convolution（下一节）中学习到的参数，而原先要预测的参数$\theta'$变成了参数$\tilde\theta$。
### Adaptive Convolution     
$ F^a = \theta' * F^P = U*\tilde\theta *_c V *F^P$     
### Feature Fusion     
$ F^{P_*} = F^P + F^a$    
作为最后的pose estimation(此外，还有一个parsing estimation)
### Classifiers     
最后在$F^{P_*}$上用一个1x1的convolution 实现一个线性分类器。同样，在$F^S$上也会实现一个线性分类器。     
## 3.3 Training and Inference
针对3.1中的损失函数，我们使用均方误差函数训练pose部分的损失，使用交叉熵函数训练parsing部分的函数，此外，PIL也可以预训练，既是可以在不同的数据集上迁移信息。